# Neeps - Easy

The "Neeps" database includes details of all teaching events in the School of Computing at Napier University in Semester 1 of the 2000/2001 academic year.

Graduated questions

The timetable database contains the following tables:

- ut_staff

id | name
---|-----
co.ACg | Cumming, Andrew
co.ACr | Crerar, Dr Alison
co.AFA | Armitage, Dr Alistair
 | ... Result truncated.

The staff table simply gives the unique identifier for each member of staff together with their full name.

- ut_student

id | name | sze | parent
---|------|----:|----------
co1.BAe | BAe | 15 | 
co1.CO | Computing 1st Year | 150 | 
co1.CO.a | Computing 1st Year a | 20 | co1.CO
co1.CO.c | Computing 1st Year c | 20 | co1.CO
co1.CO.d | Computing 1st Year d | 20 | co1.CO
 | | | ... Result truncated.

Each entry in the student table represents a group of students who follow the same timetable. The parent field is used to maintain a hierachy of groups. A student in group co1.CO.a is also in group co1.CO

- ut_room

id | name | capacity | parent
---|------|---------:|---------
co.117+118 |  | 32 | 
co.117 |  | 16 | co.117+118
co.118 |  | 16 | co.117+118
co.B7 |  | 40 | 
 | | | ... result truncated

All classrooms are listed in the room table. The parent relation is used to maintain groupings. The row co.117+118 represents a pair of rooms - sometimes these rooms are used for a single event.

- ut_event

id | modle | kind | dow | tod | duration | room
---|-------|------|-----|-----|---------:|----------
co12004.L01 | co12004 | L | Wednesday | 11:00 | 1 | cr.SMH
co12004.L02 | co12004 | L | Monday | 17:00 | 1 | cr.B13
co12004.T01 | co12004 | T | Monday | 11:00 | 2 | co.G78+G82
co12004.T02 | co12004 | T | Tuesday | 15:00 | 2 | co.B7
co12004.T03 | co12004 | T | Tuesday | 13:00 | 2 | co.G78+G82
 | | | | | | ... Result truncated.

Every event is associated with a single modle (deliberately mis-spelled version of module - which is a reserved word in one of engines). An event occurs on a particular day of the week (dow) at a particular time of day (tod). Every event has a duration specified as a whole number of hours.

- ut_attends

student | event
--------|------
co1.BAe | co12004.L01
co1.CO | co12004.L01
co1.IS | co12004.L01
co1.SE.pt | co12004.L02
co1.CO.a | co12004.T01
 | ... Result truncated.

The attends table links event to student. It realises the many to many relation.

- ut_teaches

staff | event
------|----------
co.AMn | co12004.L01
co.RK | co12004.L01
co.SRM | co12004.L01
co.RK | co12004.L02
co.SRM | co12004.L02
 | ... Result truncated.

The teaches table links event to staff.

- ut_occurs

event | week
------|-------:
co12004.L01 | 01
co12004.L01 | 02
co12004.L01 | 03
co12004.L01 | 04
co12004.L01 | 05
 | ... Result truncated.

Most events occur on weeks 01 to 13. This table records exactly which weeks are part of the plan for each event.

- ut_modle

id | name
---|---------
co12001 | Rapid Application Development
co12002 | Software Development 1A
co12003 | Professional Skills
co12004 | Applications Workshop
 | ... Result truncated.

By the 21st century, the Gradgrindians had completed their assimilation of Education. The process of "modularisation" was just one of their crimes against learning.

- ut_week

id | wkstart
--:|-------:
01 | 02/10/00
02 | 09/10/00
03 | 16/10/00
 | ...Result truncated


In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)


········


In [2]:
ut_staff = pd.read_sql_table('ut_staff', engine)
ut_student = pd.read_sql_table('ut_student', engine)
ut_event = pd.read_sql_table('ut_event', engine)
ut_room = pd.read_sql_table('ut_room', engine)
ut_attends = pd.read_sql_table('ut_attends', engine)
ut_teaches = pd.read_sql_table('ut_teaches', engine)
ut_occurs = pd.read_sql_table('ut_occurs', engine)
ut_modle = pd.read_sql_table('ut_modle', engine)
ut_week = pd.read_sql_table('ut_week', engine)


## 1.
**Give the room id in which the event co42010.L01 takes place.**

In [3]:
ut_event.loc[ut_event['id']=='co42010.L01', ['room']]


,room
102,cr.132


## 2.
**For each event in module co72010 show the day, the time and the place.**

In [4]:
ut_event.loc[ut_event['modle']=='co72010', ['dow', 'tod', 'room']]


,dow,tod,room
133,Wednesday,14:00,cr.SMH
134,Tuesday,09:00,cr.B8
135,Wednesday,09:00,co.B7
136,Tuesday,12:00,co.LB42+LB46
137,Tuesday,11:00,co.G75+G76
138,Wednesday,16:00,co.LB42+LB46
139,Thursday,10:00,co.LB42+LB46
140,Wednesday,13:00,co.117+118


## 3.
**List the names of the staff who teach on module co72010.**

In [5]:
a = (ut_event.merge(ut_teaches, left_on='id', right_on='event')
     .merge(ut_staff, left_on='staff', right_on='id'))
a.loc[a['modle']=='co72010', ['name']].drop_duplicates()


,name
29,"Cumming, Andrew"
118,"Chisholm, Ken"


## 4.
**Give a list of the staff and module number associated with events using room cr.132 on Wednesday, include the time each event starts.**

In [6]:
a = (ut_event.merge(ut_teaches, left_on='id', right_on='event')
     .merge(ut_staff, left_on='staff', right_on='id'))
a.loc[(a['room']=='cr.132') & (a['dow']=='Wednesday'), ['name', 'modle']]


,name,modle
149,"Murray, Jim",co22009
191,"Varey, Alison",co32021


## 5.
**Give a list of the student groups which take modules with the word 'Database' in the name.**

In [7]:
a = (ut_student.merge(ut_attends, left_on='id', right_on='student')
     .merge(ut_event, left_on='event', right_on='id')
     .merge(ut_modle, left_on='modle', right_on='id'))
(a.loc[a['name_y'].str.lower().str.contains('database'), ['name_x']]
 .drop_duplicates())


,name_x
169,BSc4 Computing
170,BEng4 Network and Distributing Systems
249,PgD Information Systems
250,None
251,PgD Software Engineering
252,PgD Software Technology
253,PgD Information Systems a (HCI)
254,PgD Information Systems b (DS)
255,PgD Information Systems d (BT)
257,PgD Information Systems e (OOP)
